In [2]:
import os
from model.encoder import Encoder
from model.decoder import Decoder
from model.tcvae import TCVAE
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.src.optimizers import RMSprop
from src.utils.helper import Helper
import pandas as pd

2024-02-19 16:57:37.948917: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 16:57:37.975019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 16:57:37.975050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 16:57:37.976123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 16:57:37.981668: I tensorflow/core/platform/cpu_feature_guar

In [3]:
os.environ['TFDS_DATA_DIR'] = '/mnt/sdb/home/ml/tensorflow_datasets/'

In [4]:
def get_embeddings(model, datasets):
    split = datasets['split']
    batch_size = datasets['batch_size']
    result = []
    for dataset in datasets['name']:
        data_train = tfds.load(dataset, split=[split])
        train = data_train[0].batch(batch_size).prefetch(tf.data.AUTOTUNE)
        labels = Helper.get_labels(train)

        df = model._encoder.predict(Helper.data_generator([train], method='stop'))
        df = df[0]  # reparameterize(df[0], df[1]).numpy()
        ld = df.shape[1]

        labels.index = range(0, len(labels))
        df = pd.concat([pd.DataFrame(df), labels], axis=1)
        result.append(df)

    return result, ld

In [5]:
model = tf.keras.models.load_model('../results_round_3/2024-02-16_15-25-04/model_best/')

2024-02-19 16:57:41.969320: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-19 16:57:42.056207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-19 16:57:42.056456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
datasets = {
    'name': ['icentia11k'],
    'shuffle_size': 1024,
    'batch_size': 1024,
}

splits = [
    '107', '5484', '6998', '3984', '3111', '4040', '3013', '6607', '4219', '8750', '5665', '9225',
    '8030', '9886', '1851', '1123', '3043', '3369', '6829', '10969', '3088', '9405', '9535', '4993',
    '4209', '10937', '6167', '4688', '6877', '10733', '8412', '10146', '10973', '9345', '2514', '2908',
    '5938', '5015', '9595', '8769', '4786', '2602', '7779', '2826', '1118', '3485', '2980', '10503',
    '7719', '6575', '1722', '7234', '8366', '3948', '5493', '10731', '8111', '2820', '5337', '5369',
    '4184', '9403', '9625', '303', '33', '3274', '1941', '9116', '9283', '3522', '4836', '7107', '251',
    '9071', '6899', '9733', '9440', '457', '2954', '1839', '5865', '8500', '9559', '1277', '1145', '10107',
    '9287', '8443', '9783', '9956', '10090', '3204', '6814', '4553', '6377', '5572', '1178', '5032', '1793', '4453',
]

In [1]:
ld = 12

In [ ]:
for i, split in enumerate(splits):
    print(i, '\t', 'Split:', split)
    datasets.update({'split': split})

    data_train = tfds.load('icentia11k', split=[split])
    train = data_train[0].batch(datasets['batch_size']).prefetch(tf.data.AUTOTUNE)

    encoder = Encoder(ld)
    decoder = Decoder(ld)

    model_personalization = TCVAE(encoder, decoder, {'alpha': 0.2, 'beta': 0.4, 'gamma': 0.2}, len(train))
    model_personalization.compile(optimizer=RMSprop(learning_rate=0.001))
    model_personalization.set_weights(model.weights)
    model_personalization.fit(
        Helper.data_generator([train]),
        steps_per_epoch=len(train),
        epochs=20, verbose=1,
    )
    embeddings, ld = get_embeddings(model_personalization, datasets)
    embedding = embeddings[0]
    embedding['split'] = split
    embedding.to_csv('./embedding_personalization/' + split + '.csv')

0 	 Split: 107


2024-02-19 16:57:49.657483: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-02-19 16:57:49.702110: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-19 16:57:49.894603: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/20
236/236 [==============================] - 17s 60ms/step - loss: 7.3040 - recon: 2.1283 - kl_loss: 6.4109 - mi: 4.3391 - tc: 1.6345 - dw_kl: 15.0013 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 2/20
236/236 [==============================] - 12s 53ms/step - loss: 7.3274 - recon: 2.0788 - kl_loss: 6.5136 - mi: 4.3800 - tc: 1.9071 - dw_kl: 14.2350 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 3/20
236/236 [==============================] - 13s 55ms/step - loss: 7.5491 - recon: 2.0333 - kl_loss: 6.6002 - mi: 4.5648 - tc: 2.1485 - dw_kl: 14.4205 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 4/20
236/236 [==============================] - 22s 60ms/step - loss: 7.6282 - recon: 2.0606 - kl_loss: 57.7684 - mi: 4.5350 - tc: 2.0930 - dw_kl: 14.9310 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 5/20
236/236 [==============================] - 12s 51ms/step - loss: 7.2402 - recon: 2.0070 - kl_loss: 6.6872 - mi: 4.4571 - tc: 2.0051 - dw_kl: 13.6886 - alpha: 0.

2024-02-19 17:03:40.366415: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8095925380998157109


1 	 Split: 5484
Epoch 1/20
302/302 [==============================] - 19s 60ms/step - loss: 27.2065 - recon: 20.9644 - kl_loss: 6.9740 - mi: 3.8055 - tc: 1.4778 - dw_kl: 21.4936 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 2/20
302/302 [==============================] - 18s 54ms/step - loss: 24.4868 - recon: 18.0501 - kl_loss: 6.3420 - mi: 3.8683 - tc: 1.5410 - dw_kl: 22.1514 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 3/20
302/302 [==============================] - 18s 58ms/step - loss: 24.0392 - recon: 17.8918 - kl_loss: 6.1695 - mi: 3.9683 - tc: 1.6663 - dw_kl: 20.1036 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 4/20
302/302 [==============================] - 16s 54ms/step - loss: 24.7693 - recon: 18.3530 - kl_loss: 6.0981 - mi: 3.9495 - tc: 1.7497 - dw_kl: 21.1330 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 5/20
302/302 [==============================] - 22s 59ms/step - loss: 23.9878 - recon: 17.7990 - kl_loss: 6.0578 - mi: 4.0196 - tc: 1.7494 - d

Epoch 1/20
198/198 [==============================] - 10s 43ms/step - loss: 18.7288 - recon: 14.5467 - kl_loss: 4.1868 - mi: 2.8671 - tc: 1.0400 - dw_kl: 13.8829 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 2/20
198/198 [==============================] - 8s 38ms/step - loss: 18.5235 - recon: 14.3573 - kl_loss: 4.3306 - mi: 3.1991 - tc: 0.8976 - dw_kl: 14.0415 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 3/20
198/198 [==============================] - 10s 39ms/step - loss: 17.8916 - recon: 13.6633 - kl_loss: 4.3375 - mi: 3.1476 - tc: 0.8829 - dw_kl: 14.4621 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 4/20
198/198 [==============================] - 8s 39ms/step - loss: 18.0265 - recon: 13.7477 - kl_loss: 4.3505 - mi: 3.2336 - tc: 0.9566 - dw_kl: 14.3339 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 5/20
198/198 [==============================] - 7s 37ms/step - loss: 17.9968 - recon: 13.8365 - kl_loss: 4.3703 - mi: 3.1927 - tc: 0.8715 - dw_kl: 14.1229 - alp

Epoch 1/20
277/277 [==============================] - 13s 42ms/step - loss: 44.0263 - recon: 36.6664 - kl_loss: 7.6040 - mi: 4.7098 - tc: 2.4656 - dw_kl: 22.2269 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 2/20
277/277 [==============================] - 11s 38ms/step - loss: 43.5250 - recon: 35.3915 - kl_loss: 7.5561 - mi: 4.8559 - tc: 2.6275 - dw_kl: 25.3010 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 3/20
277/277 [==============================] - 11s 39ms/step - loss: 42.6513 - recon: 35.0308 - kl_loss: 7.5974 - mi: 4.7651 - tc: 2.4350 - dw_kl: 23.5971 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 4/20
277/277 [==============================] - 11s 38ms/step - loss: 43.7728 - recon: 36.3876 - kl_loss: 7.6399 - mi: 4.7562 - tc: 2.5579 - dw_kl: 21.9384 - alpha: 0.2000 - beta: 0.8000 - gamma: 0.2000
Epoch 5/20
277/277 [==============================] - 11s 39ms/step - loss: 43.2858 - recon: 35.6368 - kl_loss: 7.6595 - mi: 4.7252 - tc: 2.6822 - dw_kl: 22.7908 - 